In [1]:
import matplotlib.pyplot as plt
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from sklearn.linear_model import LinearRegression
source = "C:\\Users\\Lidor\\Desktop\\sparkCourse\\"
CONF = 0.6

In [2]:
def searchQuery(line):
    details = line.split("\t")
    return details[0], details[1]

In [3]:
conf = SparkConf().setMaster("local[*]").setAppName("Association Rule")
try:
    sc = SparkContext(conf=conf)
except:
    sc.stop()
    sc = SparkContext(conf=conf)

In [4]:
#Database
searches = sc.textFile(f"file:///{source}user-ct-test-collection-01.txt")

In [5]:
#Delete all blank row and mapped the relevent
searches = searches.filter(lambda z: z != "").map(searchQuery)

In [6]:
searches  = searches.distinct().cache()

In [7]:
search_SUPP = searches.map(lambda x: (x[1], 1))
search_SUPP = search_SUPP.reduceByKey(lambda x, y: x + y) #(Query,SUPP(X))

In [8]:
search_support_XY = searches.join(searches)#(Tan, (X, Y)) 
search_support_XY = search_support_XY.filter(lambda z:z[1][0] != z[1][1])#delete duplicate searches

In [9]:
search_support_XY_SUPP = search_support_XY.map(lambda z: (z[1], 1)).reduceByKey(lambda x,y: x+y)#((X, Y), SUPP(XUY))

In [10]:
def doChangeInfo(info):
    X = info[0][0]
    Y = info[0][1]
    SUPPXUY = info[1]
    return X, (Y, SUPPXUY)

In [11]:
search_support_XY_SUPP = search_support_XY_SUPP.map(doChangeInfo)#(X, (Y,SUPP(XUY)))

In [12]:
RDDdict = search_SUPP.collectAsMap()#(X, SUPPX)

In [13]:
def claculate_conf(line):
    X = line[0]
    Y = line[1][0]
    SUPPXUY = line[1][1]
    SUPPX = int(RDDdict[X])
    CONF = SUPPXUY/SUPPX
    return X, Y, CONF

In [14]:
search_support_XY_CONF = search_support_XY_SUPP.map(claculate_conf)#(X, (Y,SUPP(XUY))) => #(X, Y,CONF(X=>Y))

In [15]:
#print(search_support_XY_CONF.take(100))

In [16]:
search_support_XY_CONF_WithFilter = search_support_XY_CONF.filter(lambda z: z[2] > CONF)
print(search_support_XY_CONF_WithFilter.count())
CONF = 0.8
search_support_XY_CONF_WithFilter = search_support_XY_CONF_WithFilter.filter(lambda z: z[2] > CONF)
print(search_support_XY_CONF_WithFilter.count())
CONF = 0.9
search_support_XY_CONF_WithFilter = search_support_XY_CONF_WithFilter.filter(lambda z: z[2] > CONF)
print(search_support_XY_CONF_WithFilter.count())

142535336
142506115
142505277


In [18]:
search_support_XY_CONF_WithFilter = search_support_XY_CONF.filter(lambda z: z[2] >= 0.2 and z[2] < 1.0)
print(search_support_XY_CONF_WithFilter.take(100))

[('cmt awards', 'educational displays on antioxidantsblueberries raspberries and strawberries', 0.3333333333333333), ('pet memorials', 'internet', 0.5), ('yoko ono', 'joey jordisen pics', 0.25), ('www.toys r us.com', 'www.ciricard', 0.2), ('www.metlife.com mybenefits', 'www.myspace.com ilysamamberkellydeanna', 0.5), ('university hospitals', 'address', 0.5), ('microcenter', 'wil anderson', 0.2), ('phone aol', 'millsentertainment', 0.3333333333333333), ('tic', 'right arm numb', 0.5), ('behind these hazel eyes lyrics', 'new york clip art', 0.3333333333333333), ('definition of fiction', 'emrcedes benz', 0.5), ('song of the south', 'x-men 3 movie', 0.25), ('swish', 'pier one', 0.25), ('r&s straus', 'www.video game cheats.com', 0.3333333333333333), ('chevrolet avalanche', 'www.universalorlando.com annualpass', 0.3333333333333333), ('anatomy of the eye', 'stop someone from calling you', 0.5), ('first amendment', 'german translator', 0.5), ('makeup tips', 'alice in wondrland rven riddle', 0.33